# OvenFlow: DSL para descrição de receitas

## Explicando as funções/macros de Scheme utilizadas

A macro `syntax->datum` é necessária para acessar os "dados puros" (datum) dos inputs da nossa macro higiênica quando utilizamos `syntax-case`:

In [1]:
(define-syntax inspect-syntax
  (lambda (x)
    (syntax-case x ()
      ((_ expr)
       (let* ([expr-datum (syntax->datum #'expr)])
         (format #t "Syntax Object capturado: ~s\n" #'expr)
         (format #t "Datum resultante após syntax->datum: ~s\n" expr-datum)
         #'expr)))))

(display "Testando o syntax->datum: ")
(display (inspect-syntax (+ 10 20)))

Syntax Object capturado: #<syntax:unknown file:11:25 (+ 10 20)>
Datum resultante após syntax->datum: (+ 10 20)
Testando o syntax->datum: 30

In [2]:
(define-syntax inspect-syntax
  (lambda (x)
    (syntax-case x ()
      ((_ expr ...)
       (let* ([expr-datum (syntax->datum #'(expr ...))])
         (format #t "Syntax Object capturado: ~s\n"   #'(expr ...))
         (format #t "Datum resultante após syntax->datum: ~s\n" expr-datum))))))

(display "Testando o syntax->datum: ")
(display (inspect-syntax (+ 10 20) (* 2 25)))

Syntax Object capturado: (#<syntax:unknown file:10:25 (+ 10 20)> #<syntax:unknown file:10:35 (* 2 25)>)
Datum resultante após syntax->datum: ((+ 10 20) (* 2 25))
Testando o syntax->datum: #t

Outro ponto importante é a utilização das `association lists`, que podem ser comparadas à estrutura de dicionários/hash tables em outras linguagens, como `Java` ou `C#`. A estrutura delas são como listas onde cada elemento é uma lista composta de 2 posições: primeiro o identificador e por fim o valor. Assim é uma das formas de criar essas listas, forma essa utilizada em nossa DSL:

In [3]:
(let* ((person (acons 'name "Robson" '()))
       (person-with-age (acons 'age 22 person)))
  (format #t "Person: ~a\n" person)
  (format #t "Person with age: ~a\n" person-with-age))

Person: ((name . Robson))
Person with age: ((age . 22) (name . Robson))
#t

Além disso, é necessário a função para consultar os valores dessa lista de acordo com as chaves:

In [4]:
(let* ((person (acons 'name "Robson" '()))
       (person-with-age (acons 'age 22 person)))
  (format #t "Name: ~a\n" (cdr (assv 'name person-with-age)))
  (format #t "Age: ~a\n" (cdr (assv 'age person-with-age))))

Name: Robson
Age: 22
#t

## Funções auxiliadoras

Primeiro, definimos funções para trabalhar com a estrutura de `association list` que escolhemos para tratar as receitas. Elas são compostas por 3 chaves: 
* `name`: o nome da receita.
* `ingredients`: lista de com os ingredientes da receita.
* `steps`: lista com os passos da receita.

In [5]:
; retorna uma lista com duas posições: a primeira é a lista de ingredientes, a segunda é a string com o texto do passo completo.
(define (build-step-result ingredients text-parts)
  (let* ((steps (acons 'steps (string-join (reverse text-parts) " ") '()))
         (with-ingredients (acons 'ingredients (reverse ingredients) steps)))
    with-ingredients))

(define (get-name recipe)
  (cdr (assv 'name recipe)))

(define (get-ingredients recipe)
  (cdr (assv 'ingredients recipe)))

(define (get-steps recipe)
  (cdr (assv 'steps recipe)))

(let  ((recipe-test (build-step-result '("3 ovos" "1 litro de leite") '("1 litro de leite" "com" "3 ovos" "Misturar"))))
  (format #t "Exemplo de receita: ~a\n" recipe-test)
  (format #t "Ingredientes: (~a)\n" (string-join (get-ingredients recipe-test) ", "))
  (format #t "Passos: ~a\n"(get-steps recipe-test)))

Exemplo de receita: ((ingredients 1 litro de leite 3 ovos) (steps . Misturar 3 ovos com 1 litro de leite))
Ingredientes: (1 litro de leite, 3 ovos)
Passos: Misturar 3 ovos com 1 litro de leite
#t

Também precisamos de uma função que traduza as unidades para texto, assim podemos trabalhar com unidades de medida bem definidas e conhecidas e, ainda assim, usá-las no texto de forma natural:

In [6]:
; Converte símbolos de unidade pré-definidos em texto legível.
(define (translate-unit unit-sym)
  (case unit-sym
    ; Unidades pedidas
    ((kg) "kg")
    ((g) "g")
    ((l) "l")
    ((ml) "ml")
    ((cup) " xícara(s)")
    ((teaspoon) " colher(es) de chá")
    ((tablespoon) " colher(es) de sopa")
    ((can) " lata(s)")
    ((un) " unidade(s)")

    ; Novas Unidades de TEMPO
    ((min) " minuto(s)")
    ((h) " hora(s)")
    
    ; Novas Unidades de TEMPERATURA
    ((celsius) "°C")
    ((fahrenheit) "°F")
    
    ; Fallback para qualquer outro símbolo
    (else (symbol->string unit-sym))))

(format #t "Quilos: ~a\n" (translate-unit 'kg))
(format #t "Litros: ~a\n" (translate-unit 'l))
(format #t "Unidades: ~a\n" (translate-unit 'un))
(format #t "Colher de sopa: ~a\n" (translate-unit 'tablespoon))
(format #t "Minutos: ~a\n" (translate-unit 'min))
(format #t "Graus Celsius: ~a\n" (translate-unit 'celsius))

Quilos: kg
Litros: l
Unidades:  unidade(s)
Colher de sopa:  colher(es) de sopa
Minutos:  minuto(s)
Graus Celsius: °C
#t

Então, definimos a função `process-step`: ela será responsável por identificar dentro de cada passo quais são os ingredientes e também deve retornar uma string que represente o passo em si.

In [7]:
; verifica se é compatível com (ingredient "alguma string")
(define (is-ingredient? x)
  (and (pair? x) (eq? (car x) 'ingredient)))

; Novas Funções de Predicado
(define (is-time? x)
  (and (pair? x) (eq? (car x) 'time)))

(define (is-temp? x)
  (and (pair? x) (eq? (car x) 'temp)))

; processa cada (step ...)
(define (process-step s)
(let ((elements (cdr s))) ; elimina a palavra step (primeira posição) da lista a ser processada
 (let loop ((rest elements)
            (ingredients '())
            (text-parts '())) ; text-parts é tudo aquilo que foi passado dentro do step que não é um ingredient (texto puro)
   
   (if (null? rest)
       (build-step-result ingredients text-parts)
       (let ((el (car rest)))
         (cond
          
           ; ingrediente — deve ser (ingredient "alguma string")
           ((is-ingredient? el)
            (let* ((qtd (cadr el))
                  (unit-sym (caddr el))
                  (name (cadddr el))
                  (qtd-text (number->string qtd))
                  (unit-text (translate-unit unit-sym)))
              (loop (cdr rest)
                    (cons el ingredients)
                    (cons (string-append qtd-text unit-text " de " name) text-parts))))
          
          ; NOVO: Tempo
           ((is-time? el)
            (let* ((qtd (cadr el))
                   (unit-sym (caddr el))
                   (qtd-text (number->string qtd))
                   (unit-text (translate-unit unit-sym))
                   (time-text (string-append qtd-text unit-text)))
              (loop (cdr rest)
                    ingredients ; Não adiciona aos ingredientes
                    (cons time-text text-parts))))

          ; NOVO: Temperatura
           ((is-temp? el)
            (let* ((val (cadr el))
                   (unit-sym (caddr el))
                   (val-text (number->string val))
                   (unit-text (translate-unit unit-sym))
                   (temp-text (string-append val-text unit-text)))
              (loop (cdr rest)
                    ingredients ; Não adiciona aos ingredientes
                    (cons temp-text text-parts))))
          
           ; texto literal
           ((string? el)
            (loop (cdr rest)
                  ingredients
                  (cons el text-parts)))
          
           ; qualquer outra coisa é inválida
           (else
            (error "Passo inválido" el))))))))

(process-step '(step "Misturar" (ingredient 3 un "ovos") "," (ingredient 1.5 cup "açúcar") "e" (ingredient 0.5 cup "óleo")))

((ingredients (ingredient 3 un "ovos") (ingredient 1.5 cup "açúcar") (ingredient 0.5 cup "óleo")) (steps . "Misturar 3 unidade(s) de ovos , 1.5 xícara(s) de açúcar e 0.5 xícara(s) de óleo"))

In [8]:
(process-step '(step "Assar por" (time 40 min) "a" (temp 180 celsius)))

((ingredients) (steps . "Assar por 40 minuto(s) a 180°C"))

Agora, descrevemos algumas funções auxiliadoras para ajudar na manipulação de listas:
* `take`: retorna apenas os primeiros `n` elementos da lista.
* `drop`: descarta os primeiros `n` elementos da lista.

In [9]:
(define (take lst n)
  (if (or (null? lst) (<= n 0))
      '()
      (cons (car lst) (take (cdr lst) (- n 1)))))

(define (drop lst n)
  (if (or (null? lst) (<= n 0))
      lst
      (drop (cdr lst) (- n 1))))

(define (find-step-index-by-text steps-list search-text)
  (let loop ((remaining-steps steps-list)
             (current-index 1))
    (cond
      ((null? remaining-steps)
       (error (string-append "Passo de referência não encontrado: " search-text)))
      ((string-contains (car remaining-steps) search-text)
       current-index)
      (else
       (loop (cdr remaining-steps) (+ current-index 1))))))

(format #t "Take: ~a\n" (take '(1 2 3 4 5 6 7 8 9) 3))
(format #t "Drop: ~a\n" (drop '(1 2 3 4 5 6 7 8 9) 3))
(display (find-step-index-by-text '("Passo 1" "Passo 2") "Passo 2"))

Take: (1 2 3)
Drop: (4 5 6 7 8 9)
2

Falando das modificações que podem ser feitas nas receitas, elas podem se de 3 tipos:
* `add-step-to-start`: adiciona um novo passo como primeiro na receita.
* `add-step-to-end`: adiciona um novo passo como último na receita.
* `add-step-after`: adiciona um novo passo depois do passo de número `index` na receita.
* `add-step-after-text`: adiciona um novo passo depois de um texto na receita (definido posteriormente)

Com isso, precisamos de uma função que faça essa alteração na receita:

In [10]:
(define (insert-new-steps action-sym index old-steps new-step-texts)
  (case action-sym
        ((add-step-to-start)
         (append new-step-texts old-steps))
        ((add-step-to-end)
         (append old-steps new-step-texts))
        ((add-step-after)
         (append (take old-steps index)
                 new-step-texts 
                 (drop old-steps index)))
        (else old-steps)))

(format #t "Start: ~a\n" (insert-new-steps 'add-step-to-start 2 '(3 4 5) '(1 2)))
(format #t "End: ~a\n" (insert-new-steps 'add-step-to-end 2 '(1 2 3) '(4 5)))
(format #t "After 2: ~a\n" (insert-new-steps 'add-step-after 2 '(1 2 5) '(3 4)))

Start: (1 2 3 4 5)
End: (1 2 3 4 5)
After 2: (1 2 3 4 5)
#t

E, por conta disso, precisamos de funções que manipulem as modificações e identifiquem as propriedades dela:
* `find-index-from-modification`: encontra o index a partir do qual os passos devem ser inseridos (se for o caso).
* `find-steps-from-modification`: encontra os passos da modificação que está sendo criada.
* `merge-recipes`: insere de fato a modificação na receita base.
* `build-import-body`: responsável por executar a modificação quando ela é definida por uma outra receita (algum identificador criado previamente no código).
* `build-literal-body`: responsável por executar a modificação quando ela é definida por uma sequencia de passos.

In [11]:
(define (find-index-from-modification action-sym action-args)
  (if (or (eq? action-sym 'add-step-after)
          (eq? action-sym 'add-step-after-text))
      (car action-args)
      #f))

(format #t "At start: ~a\n" (find-index-from-modification 'add-step-to-start '((step "Adicionar" (ingredient "3 cenouras médias raladas") "à mistura") (step "Assar em forno pré-aquecido a 180 graus por 40 minutos"))))
(format #t "At start: ~a\n" (find-index-from-modification 'add-step-to-start '(other-recipe)))
(format #t "After 2: ~a\n" (find-index-from-modification 'add-step-after '(2 (step "Adicionar" (ingredient "3 cenouras médias raladas") "à mistura") (step "Assar em forno pré-aquecido a 180 graus por 40 minutos"))))
(format #t "After 2: ~a\n" (find-index-from-modification 'add-step-after '(2 other-recipe)))

At start: #f
At start: #f
After 2: 2
After 2: 2
#t

In [12]:
(define (find-steps-from-modification action-sym action-args)
  (if (or (eq? action-sym 'add-step-after)
          (eq? action-sym 'add-step-after-text))
      (cdr action-args)
      action-args))

(format #t "At start: ~a\n" (find-steps-from-modification 'add-step-to-start '((step "Adicionar" (ingredient 3 un "médias raladas") "à mistura") (step "Assar em forno pré-aquecido a 180 graus por 40 minutos"))))
(format #t "At start: ~a\n" (find-steps-from-modification 'add-step-to-start '(other-recipe)))
(format #t "After 2: ~a\n" (find-steps-from-modification 'add-step-after '(2 (step "Adicionar" (ingredient 3 un "médias raladas") "à mistura") (step "Assar em forno pré-aquecido a 180 graus por 40 minutos"))))
(format #t "After 2: ~a\n" (find-steps-from-modification 'add-step-after '(2 other-recipe)))

At start: ((step Adicionar (ingredient 3 un médias raladas) à mistura) (step Assar em forno pré-aquecido a 180 graus por 40 minutos))
At start: (other-recipe)
After 2: ((step Adicionar (ingredient 3 un médias raladas) à mistura) (step Assar em forno pré-aquecido a 180 graus por 40 minutos))
After 2: (other-recipe)
#t

In [13]:
(define (merge-recipes base-recipe new-ingredients new-step-texts action-sym index)
  (let* ((old-ingredients (get-ingredients base-recipe))
         (old-steps (get-steps base-recipe))
         
         (new-ingredient-list (append new-ingredients old-ingredients))
         (new-step-list (insert-new-steps action-sym index old-steps new-step-texts)))
    
    (list (cons 'name (get-name base-recipe))
          (cons 'ingredients new-ingredient-list)
          (cons 'steps new-step-list))))

(merge-recipes '((name . "base")(ingredients . (A B C))(steps . (1 2 5))) '(D E F) '(3 4) 'add-step-after 2)

((name . "base") (ingredients D E F A B C) (steps 1 2 3 4 5))

In [14]:
 (define (build-import-body x mod-sym action-sym recipe-id-stx index-stx)
  (let ((recipe-id-datum (syntax->datum recipe-id-stx))
        (index-datum (if index-stx (syntax->datum index-stx) #f)))
    
    (datum->syntax x
      `(define ,mod-sym
         (lambda (base-recipe)
           (let* ((mod-recipe ,recipe-id-datum)
                  (new-ingredients (get-ingredients mod-recipe))
                  (new-step-texts  (get-steps mod-recipe)))
             (merge-recipes base-recipe new-ingredients new-step-texts ',action-sym ,index-datum)))))))

In [15]:
; teste de build-import-body
(let-syntax ((test-import-body (lambda (x)
                                  (syntax-case x ()
                                               ((_ mod-name (action recipe-id))
                                                (let ((mod-sym (syntax->datum #'mod-name))
                                                      (action-sym (syntax->datum #'action)))
                                                  (build-import-body x mod-sym action-sym #'recipe-id #f)))))))
  (let ((mod '((name . "mod")(ingredients . (D E))(steps . (4 5)))))
    (test-import-body teste (add-step-to-end mod))
    (teste '((name . "base")(ingredients . (A B C))(steps . (1 2 3))))))

((name . "base") (ingredients D E A B C) (steps 1 2 3 4 5))

In [16]:
(define (build-literal-body x mod-sym action-sym args-stx)
      (let ((action-args-datum (syntax->datum args-stx)))
        (datum->syntax x
          `(define ,mod-sym
             (lambda (base-recipe)
               (let* ((action-args ',action-args-datum)
                      (action-sym ',action-sym)
                      
                      (raw-index (find-index-from-modification action-sym action-args))
                      (step-forms (find-steps-from-modification action-sym action-args))
                      
                      (processed-list (map process-step step-forms))
                      (new-ingredients (apply append (map get-ingredients processed-list)))
                      (new-step-texts  (map get-steps processed-list))
                      
                      (final-index (if (eq? action-sym 'add-step-after-text)
                                       (find-step-index-by-text (get-steps base-recipe) raw-index)
                                       raw-index)) ; Se não, 'raw-index' já é o número
                                       
                      (final-action (if (eq? action-sym 'add-step-after-text)
                                        'add-step-after ; Normaliza a ação para o que 'merge-recipes' espera
                                        action-sym)))
                 
                 (merge-recipes base-recipe new-ingredients new-step-texts final-action final-index)))))))

In [17]:
; teste de build-literal-body
(let-syntax ((test-literal-body (lambda (x)
                                  (syntax-case x ()
                                               ((_ mod-name (action . args))
                                                (let ((mod-sym (syntax->datum #'mod-name))
                                                      (action-sym (syntax->datum #'action)))
                                                  (build-literal-body x mod-sym action-sym #'args)))))))
  (test-literal-body teste (add-step-after 1
                            (step "Adicionar" (ingredient 3 un "cenouras médias raladas") "à mistura e bater novamente")))
  (teste '((name . "base")(ingredients . (A B C))(steps . (1 2 5)))))

((name . "base") (ingredients (ingredient 3 un "cenouras médias raladas") A B C) (steps 1 "Adicionar 3 unidade(s) de cenouras médias raladas à mistura e bater novamente" 2 5))

Além disso, vamos precisar de funções que tratem da nossa lista de ingredientes final, fazendo a sumarização e a formatação necessárias para exibição no texto:
* `summarize-ingredients`: identifica e agrupa todas as quantidades de cada ingrediente da receita
* `format-unit-string`: formata o texto para um tipo de unidade específico (ex.: `(un . 3)` -> "3 unidade(s)").
* `format-unit-list-string`: agrupa os textos de várias unidades em uma única frase (ex.: `("3 unidade(s)" "1l" "500g")` -> "3 unidade(s), 1l e 500g").
* `format-ingredient-string`: agrupa os textos das possíveis várias unidades com o ingrediente em si (seguindo exemplo anterior: "3 unidade(s), 1l e 500g de maizena").

In [18]:
(define (update-nested-ingredient name unit qtd summarized)
  (let ((ingredient (assoc name summarized)))
    
    (if ingredient 
        ; ingrediente já está na lista
        (let* ((old-unit-list (cdr ingredient))
               (existing-unit-pair (assoc unit old-unit-list))
               (filtered-summarized (remove (lambda (pair) (equal? (car pair) name)) summarized)))
          
           (if existing-unit-pair
               ; essa unidade já consta para o ingrediente em questão
               (let* ((new-unit-qtd (+ qtd (cdr existing-unit-pair)))
                      (filtered-unit-list (remove (lambda (pair) (equal? (car pair) unit)) old-unit-list))
                      (new-unit-list (acons unit new-unit-qtd filtered-unit-list)))
                 (acons name new-unit-list filtered-summarized))
               
               ; primeira vez que essa unidade aparece para o ingrediente em questão
               (let ((new-unit-list (acons unit qtd old-unit-list)))
                 (acons name new-unit-list filtered-summarized))))
        
        ; ingrediente não está na lista
        (let ((new-unit-list (acons unit qtd '())))
          (acons name new-unit-list summarized)))))

(define (summarize-ingredients ingredient-list)
  (let loop ((remaining-list ingredient-list)
             (summary-list '()))
    (if (null? remaining-list)
        summary-list
        (let* ((current (car remaining-list))
               (qtd (cadr current))
               (unit (caddr current))
               (name (cadddr current)))
          (loop (cdr remaining-list) (update-nested-ingredient name unit qtd summary-list))))))

(summarize-ingredients '((ingredient 1 un "ovos")(ingredient 3 un "ovos")(ingredient 1 kg "ovos")(ingredient 1 un "pao")))

(("pao" (un . 1)) ("ovos" (kg . 1) (un . 4)))

In [19]:
(define (format-unit-string unit-pair)
  (let ((unit (car unit-pair))
        (qtd (cdr unit-pair)))
   (string-append (number->string qtd) (translate-unit unit))))

(format #t "~a\n" (format-unit-string '(g . 500)))
(format #t "~a\n" (format-unit-string '(cup . 2.3)))
(format #t "~a\n" (format-unit-string '(un . 1)))

500g
2.3 xícara(s)
1 unidade(s)
#t

In [20]:
(define (format-unit-list-string unit-list)
    (let join-units ((string-list unit-list))
        (let ((current (car string-list)))
            (cond
               ((null? string-list) "")
               ((null? (cdr string-list)) current)
               ((null? (cddr string-list)) (string-append current " e " (cadr string-list)))
               (else (string-append current ", " (join-units (cdr string-list))))))))

(format #t "~a\n" (format-unit-list-string '("500g")))
(format #t "~a\n" (format-unit-list-string '("500g" "2.3 xícara(s)")))
(format #t "~a\n" (format-unit-list-string '("500g" "2.3 xícara(s)" "1 unidade(s)")))

500g
500g e 2.3 xícara(s)
500g, 2.3 xícara(s) e 1 unidade(s)
#t

In [21]:
(define (format-ingredient-string ingredient-list)
  (let* ((name (car ingredient-list))
         (unit-pairs (cdr ingredient-list))
         (formatted-units (map format-unit-string unit-pairs))
         (unit-string (format-unit-list-string formatted-units)))
      (string-append unit-string " de " name)))


(format #t "~a\n" (format-ingredient-string '("ovos" (kg . 1) (g . 500) (un . 4))))
(format #t "~a\n" (format-ingredient-string '("leite" (cup . 1) (ml . 250))))
(format #t "~a\n" (format-ingredient-string '("pão" (un . 1))))

1kg, 500g e 4 unidade(s) de ovos
1 xícara(s) e 250ml de leite
1 unidade(s) de pão
#t

## Definindo as macros da DSL

Primeiro, definimos a macro `define-recipe`, que é responsável por criar uma receita base. 

In [22]:
(define-syntax define-recipe
  (lambda (x)
    (syntax-case x ()
      ((_ name step-list ...)
       (let* ((recipe-name (symbol->string (syntax->datum #'name)))
              (raw-steps   (syntax->datum #'(step-list ...))))

         (let* ((processed   (map process-step raw-steps))
                (ingredients (apply append (map get-ingredients processed)))
                (step-texts  (map get-steps processed)))
           (datum->syntax
            x
            `(define ,(syntax->datum #'name)
               '((name . ,recipe-name)
                 (ingredients . ,ingredients)
                 (steps . ,step-texts))))))))))

Depois, definimos a macro `define-modification`, que irá criar as modificações que podem ser aplicadas nas receitas.

In [23]:
(define-syntax define-modification
  (lambda (x)
    (syntax-case x ()
      ((_ mod-name (action . args))
       (let ((mod-sym (syntax->datum #'mod-name))
             (action-sym (syntax->datum #'action)))
         
         (syntax-case #'args ()
           ((index recipe-id) (and (integer? (syntax->datum #'index)) (identifier? #'recipe-id))
            (build-import-body x mod-sym action-sym #'recipe-id #'index))

           ((recipe-id) (identifier? #'recipe-id)
            (build-import-body x mod-sym action-sym #'recipe-id #f))

           (_
            (build-literal-body x mod-sym action-sym #'args))
         ))))))

Por último, temos a macro `create-recipe`, que traduzirá as receitas criadas para um texto em Markdown. 

In [24]:
(define-syntax create-recipe
  (syntax-rules ()
    ((_ final-name composition-expr)
     (let ((final-recipe composition-expr))
       (display (string-append "## " final-name "\n\n"))
       
       (display "### Ingredientes\n\n")
       (let* ((raw-ingredients (get-ingredients final-recipe))
              (summarized (summarize-ingredients raw-ingredients))
              (formatted (map format-ingredient-string summarized)))
         (let ingredients-loop ((ingredients formatted))
           (when (not (null? ingredients))
             (display (string-append "* " (car ingredients) "\n"))
             (ingredients-loop (cdr ingredients)))))
       
       (display "\n### Modo de Preparo\n\n")
       (let steps-loop ((steps (get-steps final-recipe))
                  (n 1))
         (when (not (null? steps))
           (display (string-append (number->string n) ". " (car steps) "\n"))
           (steps-loop (cdr steps) (+ n 1))))))))

## Testando receitas

In [25]:
(define-recipe bolo
  (step "Misturar" (ingredient 3 un "ovos") "," (ingredient 1.5 cup "açúcar") "e" (ingredient 0.5 cup "óleo"))
  (step "Adicionar" (ingredient 2 cup "farinha de trigo") "e misturar bem")
  (step "Assar em forno pré-aquecido a" (temp 180 celsius) "por" (time 40 min)))

(define-recipe torta
  (step "Bater no liquidificador" (ingredient 3 un "ovos") "," (ingredient 0.75 cup "óleo") "," (ingredient 1 cup "leite") "e" (ingredient 1 teaspoon "sal")))

(define-recipe calda-de-chocolate
    (step "Para a calda, misturar" (ingredient 1 can "leite condensado") "e" (ingredient 3 tablespoon "chocolate em pó") "em fogo baixo até engrossar"))

(define-modification de-cenoura
  (add-step-after 1
   (step "Adicionar" (ingredient 3 un "cenouras médias raladas") "e" (ingredient 0.5 cup "açúcar") "à mistura e bater novamente")))

(define-modification com-calda-de-chocolate
  (add-step-to-end calda-de-chocolate))

In [26]:
(display "--- Receita 1: Bolo de Cenoura com Calda ---\n")
(create-recipe "Bolo de Cenoura com Calda de Chocolate"
  (com-calda-de-chocolate (de-cenoura bolo)))

--- Receita 1: Bolo de Cenoura com Calda ---
## Bolo de Cenoura com Calda de Chocolate

### Ingredientes

* 2 xícara(s) de farinha de trigo
* 0.5 xícara(s) de óleo
* 2.0 xícara(s) de açúcar
* 3 unidade(s) de ovos
* 3 unidade(s) de cenouras médias raladas
* 3 colher(es) de sopa de chocolate em pó
* 1 lata(s) de leite condensado

### Modo de Preparo

1. Misturar 3 unidade(s) de ovos , 1.5 xícara(s) de açúcar e 0.5 xícara(s) de óleo
2. Adicionar 3 unidade(s) de cenouras médias raladas e 0.5 xícara(s) de açúcar à mistura e bater novamente
3. Adicionar 2 xícara(s) de farinha de trigo e misturar bem
4. Assar em forno pré-aquecido a 180°C por 40 minuto(s)
5. Para a calda, misturar 1 lata(s) de leite condensado e 3 colher(es) de sopa de chocolate em pó em fogo baixo até engrossar


In [27]:
(display "--- Receita 2: Torta de Cenoura ---\n")
(create-recipe "Torta de Cenoura"
  (de-cenoura torta))

--- Receita 2: Torta de Cenoura ---
## Torta de Cenoura

### Ingredientes

* 1 colher(es) de chá de sal
* 1 xícara(s) de leite
* 0.75 xícara(s) de óleo
* 3 unidade(s) de ovos
* 0.5 xícara(s) de açúcar
* 3 unidade(s) de cenouras médias raladas

### Modo de Preparo

1. Bater no liquidificador 3 unidade(s) de ovos , 0.75 xícara(s) de óleo , 1 xícara(s) de leite e 1 colher(es) de chá de sal
2. Adicionar 3 unidade(s) de cenouras médias raladas e 0.5 xícara(s) de açúcar à mistura e bater novamente


In [28]:
(display "--- Testando add-step-after-text ---\n")

(define-modification de-chocolate
  (add-step-after-text "farinha de trigo" ; <-- Nova ação
    (step "Adicionar" (ingredient 1 cup "chocolate em pó") "e misturar")))

(define-modification desperdicar
    (add-step-after-text "Assar em forno"
        (step "Jogar tudo no lixo")))

(define-modification com-fermento
  (add-step-after-text "farinha de trigo"
    (step "Por último, adicionar" (ingredient 1 tablespoon "fermento em pó") "e misturar levemente")))

(create-recipe "Bolo de Chocolate desperdiçado"
  (com-fermento(desperdicar (de-chocolate bolo))))

--- Testando add-step-after-text ---
## Bolo de Chocolate desperdiçado

### Ingredientes

* 2 xícara(s) de farinha de trigo
* 0.5 xícara(s) de óleo
* 1.5 xícara(s) de açúcar
* 3 unidade(s) de ovos
* 1 xícara(s) de chocolate em pó
* 1 colher(es) de sopa de fermento em pó

### Modo de Preparo

1. Misturar 3 unidade(s) de ovos , 1.5 xícara(s) de açúcar e 0.5 xícara(s) de óleo
2. Adicionar 2 xícara(s) de farinha de trigo e misturar bem
3. Por último, adicionar 1 colher(es) de sopa de fermento em pó e misturar levemente
4. Adicionar 1 xícara(s) de chocolate em pó e misturar
5. Assar em forno pré-aquecido a 180°C por 40 minuto(s)
6. Jogar tudo no lixo
